### Introducing the dataset
As mentioned in the video, you'll deal with stock market prices that fluctuate over time. In this exercise you've got historical prices from two tech companies (Ebay and Yahoo) in the DataFrame prices.    
You'll visualize the raw data for the two companies, then generate a scatter plot showing how the values for each company compare with one another.    
Finally, you'll add in a "time" dimension to your scatter plot so you can see how this relationship changes over time.

The data has been loaded into a DataFrame called prices.   

### Instructions
Plot the data in prices. Pay attention to any irregularities you notice.   
Generate a scatter plot with the values of Ebay on the x-axis, and Yahoo on the y-axis. Look up the symbols for both companies from the column names of the DataFrame.    

In [ ]:
# Plot the raw values over time
prices.plot()
plt.show()

# Scatterplot with one company per axis
plt.clf()
prices.plot.scatter('EBAY', 'YHOO')
plt.show()

# Scatterplot with color relating to time
plt.clf()
prices.plot.scatter('EBAY', 'YHOO', c=prices.index, 
                    cmap=plt.cm.viridis, colorbar=False)
plt.show()

### Visualizing messy data
Let's take a look at a new dataset - this one is a bit less-clean than what you've seen before.    
As always, you'll first start by visualizing the raw data. Take a close look and try to find datapoints that could be problematic for fitting models.      
The data has been loaded into a DataFrame called prices.
### Instructions
Visualize the time series data using Pandas.     
Calculate the number of missing values in each time series. Note any irregularities that you can see. What do you think they are?   

In [ ]:
# Visualize the dataset
prices.plot(legend=False)
plt.tight_layout()
plt.show()

# Count the missing values of each time series
missing_values = prices.isna().sum()
print(missing_values)

### Imputing missing values
When you have missing data points, how can you fill them in?     
In this exercise, you'll practice using different interpolation methods to fill in some missing values, visualizing the result each time. But first, you will create the function (interpolate_and_plot()) you'll use to interpolate missing data points and plot them.   
A single time series has been loaded into a DataFrame called prices.

### Instructions
Create a boolean mask for missing values and interpolate the missing values using the interpolation argument of the function.    
Interpolate using the latest non-missing value and plot the results.    
Recall that interpolate_and_plot's second input is a string specifying the kind of interpolation to use.   


In [ ]:
# Create a function we'll use to interpolate and plot
def interpolate_and_plot(prices, interpolation):

    # Create a boolean mask for missing values
    missing_values = prices.isna()

    # Interpolate the missing values
    prices_interp = prices.interpolate(interpolation)

    # Plot the results, highlighting the interpolated values in black
    fig, ax = plt.subplots(figsize=(10, 5))
    prices_interp.plot(color='k', alpha=.6, ax=ax, legend=False)
    
    # Now plot the interpolated values on top in red
    prices_interp[missing_values].plot(ax=ax, color='r', lw=3, legend=False)
    plt.show()

In [ ]:
# Interpolate using the latest non-missing value
interpolation_type = 'zero'
interpolate_and_plot(prices, interpolation_type)

In [ ]:
# Interpolate linearly
interpolation_type = 'linear'
interpolate_and_plot(prices, interpolation_type)

In [ ]:
# Interpolate with a quadratic function
interpolation_type = 'quadratic'
interpolate_and_plot(prices, interpolation_type)

### Transforming raw data
In the last chapter, you calculated the rolling mean. In this exercise, you will define a function that calculates the percent change of the latest data point from the mean of a window of previous data points. This function will help you calculate the percent change over a rolling window.    
This is a more stable kind of time series that is often useful in machine learning.

### Instructions
Define a percent_change function that takes an input time series and does the following:    
Extract all but the last value of the input series (assigned to previous_values) and the only the last value of the timeseries ( assigned to last_value)    
Calculate the percentage difference between the last value and the mean of earlier values.   
Using a rolling window of 20, apply this function to prices, and visualize it using the given code.   

In [ ]:
# Your custom function
def percent_change(series):
    # Collect all *but* the last value of this window, then the final value
    previous_values = series[:-1]
    last_value = series[-1]

    # Calculate the % difference between the last value and the mean of earlier values
    percent_change = (last_value - np.mean(previous_values)) / np.mean(previous_values)
    return percent_change

# Apply your custom function and plot
prices_perc = prices.rolling(20).aggregate(percent_change)
prices_perc.loc["2014":"2015"].plot()
plt.show()

### Handling outliers
In this exercise, you'll handle outliers - data points that are so different from the rest of your data, that you treat them differently from other "normal-looking" data points. You'll use the output from the previous exercise (percent change over time) to detect the outliers. First you will write a function that replaces outlier data points with the median value from the entire time series.   
### Instructions
Define a function that takes an input series and does the following:     
Calculates the absolute value of each datapoint's distance from the series mean, then creates a boolean mask for datapoints that are three times the standard deviation from the mean.    
Use this boolean mask to replace the outliers with the median of the entire series.    
Apply this function to your data and visualize the results using the given code.   

In [ ]:
def replace_outliers(series):
    # Calculate the absolute difference of each timepoint from the series mean
    absolute_differences_from_mean = np.abs(series - np.mean(series))
    
    # Calculate a mask for the differences that are > 3 standard deviations from zero
    this_mask = absolute_differences_from_mean > (np.std(series) * 3)
    
    # Replace these values with the median accross the data
    series[this_mask] = np.nanmedian(series)
    return series
    
# Apply your preprocessing function to the timeseries and plot the results
prices_perc = prices_perc.aggregate(replace_outliers)
prices_perc.loc["2014":"2015"].plot()
plt.show()

### Engineering multiple rolling features at once
Now that you've practiced some simple feature engineering, let's move on to something more complex. You'll calculate a collection of features for your time series data and visualize what they look like over time. This process resembles how many other time series models operate.
### Instructions
Define a list consisting of four features you will calculate: the minimum, maximum, mean, and standard deviation (in that order).    
Using the rolling window (prices_perc_rolling) we defined for you, calculate the features from features_to_calculate.    
Plot the results over time, along with the original time series using the given code.    

In [ ]:
# Define a rolling window with Pandas, excluding the right-most datapoint of the window
prices_perc_rolling = prices_perc.rolling(20, min_periods=5, closed='right')

# Define the features you'll calculate for each window
features_to_calculate = [np.min, np.max, np.mean, np.std]

# Calculate these features for your rolling window object
features = prices_perc_rolling.aggregate(features_to_calculate)

# Plot the results
ax = features.loc[:"2011-01"].plot()
prices_perc.loc[:"2011-01"].plot(ax=ax, color='k', alpha=.2, lw=3)
ax.legend(loc=(1.01, .6))
plt.show()

### Percentiles and partial functions
In this exercise, you'll practice how to pre-choose arguments of a function so that you can pre-configure how it runs. You'll use this to calculate several percentiles of your data using the same percentile() function in numpy.

### Instructions
Import partial from functools.   
Use the partial() function to create several feature generators that calculate percentiles of your data using a list comprehension.   
Using the rolling window (prices_perc_rolling) we defined for you, calculate the quantiles using percentile_functions.   
Visualize the results using the code given to you.

In [ ]:
# Import partial from functools
from functools import partial
percentiles = [1, 10, 25, 50, 75, 90, 99]

# Use a list comprehension to create a partial function for each quantile
percentile_functions = [partial(np.percentile, q=percentile) for percentile in percentiles]

# Calculate each of these quantiles on the data using a rolling window
prices_perc_rolling = prices_perc.rolling(20, min_periods=5, closed='right')
features_percentiles = prices_perc_rolling.aggregate(percentile_functions)

# Plot a subset of the result
ax = features_percentiles.loc[:"2011-01"].plot(cmap=plt.cm.viridis)
ax.legend(percentiles, loc=(1.01, .5))
plt.show()

### Using "date" information
It's easy to think of timestamps as pure numbers, but don't forget they generally correspond to things that happen in the real world. That means there's often extra information encoded in the data such as "is it a weekday?" or "is it a holiday?". This information is often useful in predicting timeseries data.    
In this exercise, you'll extract these date/time based features. A single time series has been loaded in a variable called prices.

### Instructions

Calculate the day of the week, week number in a year, and month number in a year.   
Add each one as a column to the prices_perc DataFrame, under the names day_of_week, week_of_year and month_of_year, respectively.   

In [ ]:
# Extract date features from the data, add them as columns
prices_perc['day_of_week'] = prices_perc.index.weekday
prices_perc['week_of_year'] = prices_perc.index.week
prices_perc['month_of_year'] = prices_perc.index.month

# Print prices_perc
print(prices_perc)